In [1]:
!nvidia-smi

Tue Nov  5 18:47:07 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.90                 Driver Version: 565.90         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8              3W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import os
HOME = os.getcwd()
print(HOME)

c:\Users\pspkm\Desktop\QDC Lab Code


In [23]:
SOURCE_VIDEO_PATH = r'C:\Users\pspkm\Desktop\QDC Lab Code\video.mp4'

In [5]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.145  Python-3.9.19 torch-1.12.1 CPU (AMD Ryzen 5 3550H with Radeon Vega Mobile Gfx)
Setup complete  (8 CPUs, 13.9 GB RAM, 268.0/418.3 GB disk)


In [31]:
from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.1.0


In [7]:
MODEL = r'C:\Users\pspkm\Desktop\QDC Lab Code\best.pt'

In [9]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


In [10]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [0]

In [11]:
import supervision as sv
import numpy as np

In [30]:
# create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator(thickness=4)
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for confidence, class_id in zip(detections.confidence, detections.class_id)
]

# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

AttributeError: module 'supervision' has no attribute 'get_video_frames_generator'

## Predict and annotate whole video

In [ ]:
# settings
LINE_START = sv.Point(10, 700)
LINE_END = sv.Point(400-50, 700)

TARGET_VIDEO_PATH = f"{HOME}/washer_detector.mp4"

In [ ]:
sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

VideoInfo(width=400, height=800, fps=20, total_frames=2934)

In [ ]:
import numpy as np
import cv2
import supervision as sv
import csv
import pandas as pd

# Define color constants
GREEN = (0, 255, 0)
RED = (0, 0, 255)  # Red in BGR format

# Initialize variables for storing washer information
washer_info = []
trackers = []
next_id = 1

# Define a function to calculate the Euclidean distance between two points
def euclidean_distance(pt1, pt2):
    return np.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)

# Define a function to update trackers with new detections
def update_trackers(detections, frame):
    global next_id

    for det in detections:
        bbox_center = ((det[0] + det[2]) / 2, (det[1] + det[3]) / 2)
        matched = False

        # Attempt to match the detection to an existing tracker
        for tracker in trackers:
            if euclidean_distance(tracker['center'], bbox_center) < 50:
                tracker['bbox'] = det
                tracker['center'] = bbox_center
                tracker['frame_count'] = 0  # Reset the frame count after an update
                matched = True
                break

        # If no match is found, create a new tracker
        if not matched:
            trackers.append({
                'id': next_id,
                'bbox': det,
                'center': bbox_center,
                'frame_count': 0
            })
            next_id += 1

    # Remove old trackers that haven't been updated for a while
    trackers[:] = [t for t in trackers if t['frame_count'] < 5]

# Update the callback function
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    global washer_info
    global trackers

    annotated_frame = frame.copy()

    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    detections = detections[np.isin(detections.class_id, selected_classes)]

    update_trackers(detections.xyxy, frame)

    # Annotate each tracked object
    for tracker in trackers:
        tracker['frame_count'] += 1
        bbox = tracker['bbox']

        # Calculate the width and height of the bounding box
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]

        # Approximate the diameter as the diagonal of the bounding box
        diameter = np.sqrt(width**2 + height**2)

        diameter_mm = diameter*0.3978 - 1.6812

        # Set the color based on the diameter
        color = GREEN if 26.5 <= diameter_mm <= 27.5 else RED

        annotated_frame = cv2.rectangle(
            img=annotated_frame,
            pt1=(int(bbox[0]), int(bbox[1])),
            pt2=(int(bbox[2]), int(bbox[3])),
            color=color,
            thickness=4
        )

        # Add the washer number and diameter at the top of the bounding box
        label = f"Washer {tracker['id']}: {diameter_mm:.2f} mm"
        annotated_frame = cv2.putText(
            img=annotated_frame,
            text=label,
            org=(int(bbox[0]), int(bbox[1]) - 10),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.5,
            color=color,
            thickness=2,
            lineType=cv2.LINE_AA
        )

        # Save washer information
        washer_info.append({
            'Washer ID': tracker['id'],
            'Diagonal (px)': diameter,
            'Diameter (mm)': diameter_mm
        })

    return annotated_frame

# Process the whole video
sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback
)



In [ ]:
import pandas as pd

# Create a DataFrame from the washer_info list
df = pd.DataFrame(washer_info)

# Group by 'Washer ID' and count the occurrences of each washer
washer_counts = df.groupby('Washer ID').size().reset_index(name='Frame Count')

# Filter for washers appearing in more than 200 frames
filtered_washers = washer_counts[washer_counts['Frame Count'] >= 200]

# Merge filtered washers with original DataFrame to get washer information
merged_df = pd.merge(filtered_washers, df, on='Washer ID')

# Group the merged DataFrame by 'Washer ID' to calculate the average diagonal and diameter
average_df = merged_df.groupby('Washer ID').agg({
    'Diagonal (px)': 'mean',
    'Diameter (mm)': 'mean'
}).reset_index()

# Merge the average_df with the filtered_washers to retain the Frame Count
average_df = pd.merge(average_df, filtered_washers, on='Washer ID')

# Save the results to a CSV file
average_df.to_csv('filtered_washer_info.csv', index=False)

# Print a summary (Optional)
print(f"Processed video saved to: {TARGET_VIDEO_PATH}")
print(f"CSV file saved to: filtered_washer_info.csv")
print(f"Number of washers: {len(average_df)}")


Processed video saved to: /content/washer_detector.mp4
CSV file saved to: filtered_washer_info.csv
Number of washers: 14
